# Importamos librerías

In [1]:
import tensorflow as tf
import pathlib
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import keras

In [2]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout, BatchNormalization

In [3]:
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

# Importamos los datos

In [4]:
path = 'D:/Documentos de trabajo/Maestría/Tesis Maestría/Trabajo Python Tesis/Github Tesis/Base de datos'

## Datos de entrenamiento

### Datos

In [5]:
campvectrain = np.load(path + '/campvectrain.npy')

In [6]:
np.shape(campvectrain)

(7000, 2, 100, 100)

### Coeficientes

In [7]:
coefcampvectrain = np.load(path + '/coefcampvectrain.npy')

In [8]:
np.shape(coefcampvectrain)

(7000, 7)

## Datos de prueba

### Datos

In [9]:
campvectest = np.load(path + '/campvectest.npy')

In [10]:
np.shape(campvectest)

(2000, 2, 100, 100)

### Coeficientes

In [11]:
coefcampvectest = np.load(path + '/coefcampvectest.npy')

In [12]:
np.shape(coefcampvectest)

(2000, 7)

## Datos de validación

### Datos

In [13]:
campvecval = np.load(path + '/campvecval.npy')

In [14]:
np.shape(campvecval)

(1000, 2, 100, 100)

### Coeficienes

In [15]:
coefcampvecval = np.load(path + '/coefcampvecval.npy')

In [16]:
np.shape(coefcampvecval)

(1000, 7)

# Unimos coeficientes con los datos

In [17]:
train_dataset = tf.data.Dataset.from_tensor_slices((campvectrain, coefcampvectrain))
test_dataset = tf.data.Dataset.from_tensor_slices((campvectest, coefcampvectest))
val_dataset = tf.data.Dataset.from_tensor_slices((campvecval, coefcampvecval))

In [18]:
train_dataset

<TensorSliceDataset element_spec=(TensorSpec(shape=(2, 100, 100), dtype=tf.float64, name=None), TensorSpec(shape=(7,), dtype=tf.int32, name=None))>

# Mezclar y procesar por lotes los conjuntos de datos

In [19]:
BATCH_SIZE = 10
SHUFFLE_BUFFER_SIZE = 10

train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset = test_dataset.batch(BATCH_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE)

# Creamos función de costo

In [20]:
@tf.function
def custom_loss(y_true,y_pred):
    campvec = []
    # constantes
    a1 = tf.cast(y_true[:,0], tf.float64)
    a2 = tf.cast(y_true[:,1], tf.float64)
    b1 = tf.cast(y_true[:,2], tf.float64)
    b2 = tf.cast(y_true[:,3], tf.float64)
    d1 = tf.cast(y_true[:,4], tf.float64)
    d2 = tf.cast(y_true[:,5], tf.float64)
    n = tf.cast(y_true[:,6], tf.float64)
    
    def f(t, y):
        
        # asignar a cada ODE a un elemento de vector
        X = y[0]
        X = tf.cast(X, tf.float64)
        Y = y[1]
        Y = tf.cast(Y, tf.float64)
        
        # definimos cada ODE
        dX_dt = a1 / (1 + tf.math.pow(Y, n)) - d1 * X + b1
        dY_dt = a2 / (1 + tf.math.pow(X, n)) - d2 * Y + b2
        return [dX_dt, dY_dt]

    x_range_neg = 0
    x_range_pos = 50
    y_range_neg = 0
    y_range_pos = 50
    y1 = tf.linspace(x_range_neg, x_range_pos, 100)
    y2 = tf.linspace(y_range_neg, y_range_pos, 100)
    Y1, Y2 = tf.meshgrid(y1, y2)
    t = 0
    u, v = tf.zeros(Y1.shape), tf.zeros(Y2.shape)
    pruebas1, pruebas2 = f(t,[0,0])
    u, v = f(t, [Y1, Y2])
    
    
    M = tf.sqrt(tf.square(u) + tf.square(v))
    u /= M
    v /= M
# ----------------------------------------------------------------------------
    campvec2 = []
    # constantes
    a12 = tf.cast(y_pred[:,0], tf.float64)
    a22 = tf.cast(y_pred[:,1], tf.float64)
    b12 = tf.cast(y_pred[:,2], tf.float64)
    b22 = tf.cast(y_pred[:,3], tf.float64)
    d12 = tf.cast(y_pred[:,4], tf.float64)
    d22 = tf.cast(y_pred[:,5], tf.float64)
    n2 = tf.cast(y_pred[:,6], tf.float64)
    
    def f2(t2, ye2):
        
        # asignar a cada ODE a un elemento de vector
        X2 = ye2[0]
        X2 = tf.cast(X2, tf.float64)
        Y2 = ye2[1]
        Y2 = tf.cast(Y2, tf.float64)
       
        # definimos cada ODE
        dX2_dt = a12 / (1 + tf.math.pow(Y2, n2)) - d12 * X2 + b12
        dY2_dt = a22 / (1 + tf.math.pow(X2, n2)) - d22 * Y2 + b22
        
        return [dX2_dt, dY2_dt]
    

    u2, v2 = tf.zeros(Y1.shape), tf.zeros(Y2.shape)
    u2, v2 = f2(t, [Y1, Y2])
    M2 = tf.sqrt(tf.square(u2) + tf.square(v2))
    u2 /= M2
    v2 /= M2

    campvec =tf.experimental.numpy.append([u] ,[v], axis=0)
    campvec2 = tf.experimental.numpy.append([u2] ,[v2], axis=0)
    resta = campvec-campvec2
    #cosine_sim = tf.keras.losses.cosine_similarity(campvec, campvec2)
    loss = tf.reduce_mean(tf.square(resta))
    return loss

# Funcion de costo prueba

In [29]:
def custom_loss(y_true,y_pred):
    def meshgrid(x, y):
        xx = tf.repeat(tf.expand_dims(x, 1), y.shape[0], axis=1)
        yy = tf.repeat(tf.expand_dims(y, 0), x.shape[0], axis=0)
        return xx, yy
    
    # Constantes
    a1 = tf.cast(y_true[:,0], tf.float64)
    a2 = tf.cast(y_true[:,1], tf.float64)
    b1 = tf.cast(y_true[:,2], tf.float64)
    b2 = tf.cast(y_true[:,3], tf.float64)
    d1 = tf.cast(y_true[:,4], tf.float64)
    d2 = tf.cast(y_true[:,5], tf.float64)
    n = tf.cast(y_true[:,6], tf.float64)

    def f(t, y):
        # Asignar a cada ODE a un elemento de vector
        X = y[0]
        X = tf.cast(X, tf.float64)
        Y = y[1]
        Y = tf.cast(Y, tf.float64)

        # Definimos cada ODE
        dX_dt = a1 / (1 + tf.math.pow(Y, n)) - d1 * X + b1
        dY_dt = a2 / (1 + tf.math.pow(X, n)) - d2 * Y + b2
        return [dX_dt, dY_dt]

    x_range_neg = 0
    x_range_pos = 50
    y_range_neg = 0
    y_range_pos = 50
    y1 = tf.linspace(x_range_neg, x_range_pos, 100)
    y2 = tf.linspace(y_range_neg, y_range_pos, 100)
    Y1, Y2 = meshgrid(y1, y2)
    t = 0
    u, v = tf.zeros(Y1.shape), tf.zeros(Y2.shape)
    pruebas1, pruebas2 = f(t,[0,0])
    u, v = f(t, [Y1, Y2])

    M = tf.sqrt(tf.square(u) + tf.square(v))
    u /= M
    v /= M
    
# ----------------------------------------------------------------------------
    campvec2 = []
    # constantes
    a12 = tf.cast(y_pred[:,0], tf.float64)
    a22 = tf.cast(y_pred[:,1], tf.float64)
    b12 = tf.cast(y_pred[:,2], tf.float64)
    b22 = tf.cast(y_pred[:,3], tf.float64)
    d12 = tf.cast(y_pred[:,4], tf.float64)
    d22 = tf.cast(y_pred[:,5], tf.float64)
    n2 = tf.cast(y_pred[:,6], tf.float64)
    
    def f2(t2, ye2):
        
        # asignar a cada ODE a un elemento de vector
        X2 = ye2[0]
        X2 = tf.cast(X2, tf.float64)
        Y2 = ye2[1]
        Y2 = tf.cast(Y2, tf.float64)
       
        # definimos cada ODE
        dX2_dt = a12 / (1 + tf.math.pow(Y2, n2)) - d12 * X2 + b12
        dY2_dt = a22 / (1 + tf.math.pow(X2, n2)) - d22 * Y2 + b22
        
        return [dX2_dt, dY2_dt]
    

    u2, v2 = tf.zeros(Y1.shape), tf.zeros(Y2.shape)
    u2, v2 = f2(t, [Y1, Y2])
    M2 = tf.sqrt(tf.square(u2) + tf.square(v2))
    u2 /= M2
    v2 /= M2

    campvec =tf.experimental.numpy.append([u] ,[v], axis=0)
    campvec2 = tf.experimental.numpy.append([u2] ,[v2], axis=0)
    resta = campvec-campvec2
    #cosine_sim = tf.keras.losses.cosine_similarity(campvec, campvec2)
    loss = tf.reduce_mean(tf.square(resta))
    print(loss)
    return loss

# Creación del modelo

In [30]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(2,100,100)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(7)
])


In [31]:
model = Sequential()
model.add(Flatten(input_shape=(2,100,100)))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64,activation = 'relu'))
model.add(Dense(128,activation = 'relu'))
model.add(Dense(64,activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(128,activation = 'relu'))
model.add(Dense(64,activation = 'relu'))

model.add(Dense(7))

In [32]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_3 (Flatten)         (None, 20000)             0         
                                                                 
 dense_17 (Dense)            (None, 128)               2560128   
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 dense_18 (Dense)            (None, 64)                8256      
                                                                 
 dense_19 (Dense)            (None, 128)               8320      
                                                                 
 dense_20 (Dense)            (None, 64)                8256      
                                                                 
 dropout_5 (Dropout)         (None, 64)               

In [33]:
keras.utils.plot_model(model,show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


# Entrenamos el modelo

In [34]:
model.compile(optimizer='Adam',
              loss= custom_loss,
              metrics=['accuracy'])

In [35]:
def scheduler(epoch, lr):
  if epoch < 500:
    return lr
  else:
    return 1e-3 * 0.99 ** epoch

In [36]:
val_epochs = 100
tf.compat.v1.global_variables_initializer()

early_stop = tf.keras.callbacks.EarlyStopping( monitor = 'val_loss', patience = 800,verbose = 1, 
                                              restore_best_weights = True)

reduce_lr = tf.keras.callbacks.LearningRateScheduler(scheduler)
history = model.fit(train_dataset, validation_data=val_dataset, epochs= val_epochs, callbacks=[reduce_lr, early_stop])

Epoch 1/100
Tensor("custom_loss/Mean:0", shape=(), dtype=float64)
Tensor("custom_loss/Mean:0", shape=(), dtype=float64)


InvalidArgumentError: Graph execution error:

Detected at node 'custom_loss/mul_2' defined at (most recent call last):
    File "C:\Users\rafae\anaconda3\envs\tfgpu\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\rafae\anaconda3\envs\tfgpu\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\rafae\anaconda3\envs\tfgpu\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\rafae\anaconda3\envs\tfgpu\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\Users\rafae\anaconda3\envs\tfgpu\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "C:\Users\rafae\anaconda3\envs\tfgpu\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\rafae\anaconda3\envs\tfgpu\lib\asy

# Analizamos accuracy y loss

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(val_epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

# Guardamos el modelo

In [ ]:
path_to_save = 'D:/Documentos de trabajo/Maestría/Tesis Maestría/Trabajo Python Tesis/Github Tesis/Modelos'

In [ ]:
model.save(path_to_save + '/campovectorial.h5')

In [ ]:
keras.experimental.export_saved_model(model, path_to_save + 'prueba')

# Importamos el modelo

In [ ]:
new_model = keras.models.load_model('../Modelos/campovectorial.h5')

In [ ]:
new_model = keras.experimental.load_from_saved_model('../Modelos/prueba')


# Probamos el modelo con datos nuevos

## Creamos nuevos datos

In [ ]:
import random
from scipy.integrate import solve_ivp

campvec = []
coef = []
contador = 0
for i in range(0, 1):
    # constantes
    a1 = random.randint(0, 10)
    a2 = random.randint(0, 10)
    b1 = random.randint(0, 10)
    b2 = random.randint(0, 10)
    d1 = random.randint(0, 10)
    d2 = random.randint(0, 10)
    n = random.randint(0, 5)

    coef1 = [a1, a2, b1, b2, d1, d2, n]
    coef.append(coef1)

    def f(t, y):

        # asignar a cada ODE a un elemento de vector
        X = y[0]
        Y = y[1]

        # definimos cada ODE
        # dX_dt=a1/(1+Y**n)-d1*X+b1
        # dY_dt=a2/(1+X**n)-d2*Y+b2
        dX_dt = a1/(1+Y**n)-d1*X+b1
        dY_dt = a2/(1+X**n)-d2*Y+b1

        return [dX_dt, dY_dt]

    x_range_neg = 0
    x_range_pos = 50
    y_range_neg = 0
    y_range_pos =50

    y1 = np.linspace(x_range_neg, x_range_pos, 100)
    y2 = np.linspace(y_range_neg, y_range_pos, 100)

    Y1, Y2 = np.meshgrid(y1, y2)
    
    t1 = 0

    u, v = np.zeros(Y1.shape), np.zeros(Y2.shape)

    NI, NJ = Y1.shape

    u1, v1 = f(t1, [Y1, Y2])
    u, v = f(t1, [Y1, Y2])
    M = (np.hypot(u, v))
    u /= M
    v /= M

    campvecci = np.append([u1], [v1], axis=0)

    campvec = campvecci
    print('--------------------------------------------')
    print(coef1)
    contador = contador+1
    print(contador)
    
campvec = np.array(campvec)
coef = np.array(coef)


In [ ]:
campvecnone = campvec[None, :]

## Predecimos con los datos nuevos

In [ ]:
new_predictions = new_model.predict(campvecnone)
print(new_predictions)

In [ ]:
new_predictions = (model.predict(campvecnone))
print(new_predictions)


## Graficamos con los coeficientes reales

In [ ]:
y1 = np.linspace(0, 50, 100)
y2 = np.linspace(0, 50, 100)
Y1, Y2 = np.meshgrid(y1, y2)
t1 = 0
u, v = np.zeros(Y1.shape), np.zeros(Y2.shape)
NI, NJ = Y1.shape        
u,v = np.array(campvec)[0,:,:], np.array(campvec)[1,:,:]
M = (np.hypot(u,v))
u /= M
v /= M

plt.figure(figsize=(10,10))
Q = plt.quiver(Y1, Y2, u, v, M, angles='xy')
plt.title('Campo vectorial espacio fase Toggle Swich', fontsize=20)
plt.xlim([0, 50])
plt.ylim([0, 50])
plt.xlabel('Concentración X')
plt.ylabel('Concentración Y')

In [ ]:
np.shape(new_predictions)

## Graficamos con los coeficientes predecidos

In [ ]:
a1 = new_predictions[0,0]
a2 = new_predictions[0,1]
b1 = new_predictions[0,2]
b2 = new_predictions[0,3]
d1 = new_predictions[0,4]
d2 = new_predictions[0,5]
n = new_predictions[0,6]
 
def f(t, y):

    # asignar a cada ODE a un elemento de vector
    X = y[0]
    Y = y[1]

    # definimos cada ODE
    dX_dt = a1/(1+Y**n)-d1*X+b1
    dY_dt = a2/(1+X**n)-d2*Y+b1

    return [dX_dt, dY_dt]

x_range_neg = 0
x_range_pos = 50
y_range_neg = 0
y_range_pos =50

y1 = np.linspace(x_range_neg, x_range_pos, 100)
y2 = np.linspace(y_range_neg, y_range_pos, 100)

Y1, Y2 = np.meshgrid(y1, y2)
    
t1 = 0

u, v = np.zeros(Y1.shape), np.zeros(Y2.shape)

NI, NJ = Y1.shape

u1, v1 = f(t1, [Y1, Y2])
u, v = f(t1, [Y1, Y2])
M = (np.hypot(u, v))
u /= M
v /= M
    
plt.figure(figsize=(10, 10))
Q = plt.quiver(Y1, Y2, u, v, M, angles='xy')
plt.title('Campo vectorial espacio fase Toggle Swich', fontsize=20)
plt.xlim([x_range_neg, x_range_pos])
plt.ylim([y_range_neg, y_range_pos])
plt.xlabel('Concentración X')
plt.ylabel('Concentración Y')